In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
sub = pd.read_csv("../submissions/lgb_mean_ensemble_smo.csv")

sample_sub = pd.read_csv("../submissions/sample_submission.csv")
sample_sub = sample_sub[["meter_id"]]
sub = sample_sub.merge(sub, on=["meter_id"])

print(sub.shape)
sub.head()

(3248, 13)


,meter_id,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,112.0,117.0,113.0,110.0,104.0,103.0,105.0,104.0,103.0,106.0,107.0,111.0
1,0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da,480.0,501.0,484.0,462.0,434.0,420.0,426.0,424.0,424.0,443.0,452.0,468.0
2,0x4a1ed36825360a058cec2bdd409fc2459e1ce54f,316.0,329.0,317.0,304.0,288.0,282.0,286.0,284.0,283.0,295.0,300.0,310.0
3,0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407,458.0,477.0,463.0,446.0,420.0,408.0,414.0,412.0,412.0,428.0,434.0,447.0
4,0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f,774.0,780.0,756.0,706.0,657.0,615.0,615.0,617.0,631.0,684.0,724.0,762.0


In [3]:
df_train = pd.read_pickle("../input/df_train.pkl")

df_month_group = df_train.groupby(["meter_id", "month"])["meter_reading"].sum().unstack()
month_cols = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
df_train_month = pd.DataFrame(columns=["meter_id"] + month_cols)

df_train_month["meter_id"] = df_month_group.index.values
df_train_month[month_cols] = df_month_group.values.round(3)

df_train_month = df_train_month.append({"meter_id": "0x81fa8eddb2b09393d3719984ca5520cb50f45efd"}, ignore_index=True)
df_train_month.fillna(0, inplace=True)

print(df_train_month.shape)
df_train_month.head()

(3248, 13)


,meter_id,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,0x0001f1c389823f953b2eaee0a61c33539744da0c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129.613,106.645,108.575,70.879,80.144
1,0x000f697092696c27b0bb489ea4c11280ef72ab82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,438.427,585.665,476.649
2,0x0019979ee12c59accd24d1c83291528ced1bb734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,174.890,164.460,190.814,178.122
3,0x001d6227832325ab6167b82d6d7175d254f57c3a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,141.910,119.231
4,0x005958406351bb29580475df698b5f1070096397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,286.262


In [4]:
df_month_group = df_train.groupby(["meter_id", "month"])["meter_reading"].sum().unstack()
month_cols = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
df_train_month = pd.DataFrame(columns=["meter_id"] + month_cols)

df_train_month["meter_id"] = df_month_group.index.values
df_train_month[month_cols] = df_month_group.values.round(3)

df_train_month = df_train_month.append({"meter_id": "0x81fa8eddb2b09393d3719984ca5520cb50f45efd"}, ignore_index=True)
df_train_month.fillna(0, inplace=True)

print(df_train_month.shape)
df_train_month.tail()

(3248, 13)


,meter_id,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
3243,0xffbdc1d9209f45668fb0be65b6d50cb999890ccb,0.000,0.000,719.776,711.522,540.102,475.356,436.678,444.550,433.127,444.769,557.445,451.334
3244,0xffcbcdc7c3ce3451252b84c2aaf6f40d652d16d5,0.000,0.000,0.000,376.099,296.066,256.257,261.641,266.071,265.127,271.185,405.563,381.813
3245,0xffe91f021d145e1560fcad8c3af5ac0ef9d76a02,0.000,381.811,326.751,408.771,171.977,190.987,176.676,181.973,179.785,207.632,317.345,297.997
3246,0xfff895258c21f1a58fc06538173d02b621021ad4,208.599,215.099,232.268,212.296,180.257,205.302,210.694,211.191,205.619,201.814,246.600,183.019
3247,0x81fa8eddb2b09393d3719984ca5520cb50f45efd,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [5]:
# VISUALIZE MONTH PREDICTIONS
for idx, meter_idx in enumerate(tqdm(sub["meter_id"].values)):

    df_meter_idx = df_train[df_train.meter_id == meter_idx]

    fig = plt.figure(figsize=(21,9))
    plt.plot(range(12), df_train_month[df_train_month.meter_id == meter_idx].values.reshape([-1])[1:], lw=5, label="train")
    plt.plot(range(12), df_train_month[df_train_month.meter_id == meter_idx].values.reshape([-1])[1:], ".b", ms=25)

    plt.plot(range(12, 24), sub[sub.meter_id == meter_idx].values.reshape([-1])[1:], lw=5, label="prediction")
    plt.plot(range(12, 24), sub[sub.meter_id == meter_idx].values.reshape([-1])[1:], ".g", ms=25)

    plt.grid()
    plt.legend(fontsize=15)
    plt.title(f"{idx}-{meter_idx}", fontsize=25)

    plt.savefig(f"../eda/ensemble_visual_check/{idx}-{meter_idx}.png")